<a href="https://colab.research.google.com/github/karthik-palani/machine_learning/blob/master/R6_Internal_Lab_UpdatedTF2_Prices_Iris.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Linear Classifier in TensorFlow 
Using Low Level API in Eager Execution mode

### Load tensorflow

In [0]:
import tensorflow as tf

In [0]:
#Enable Eager Execution if using tensflow version < 2.0
#From tensorflow v2.0 onwards, Eager Execution will be enabled by default


In [2]:
tf.__version__

'1.13.1'

In [0]:
tf.enable_eager_execution()

### Collect Data

In [0]:
#from google.colab import drive
#drive.mount('/gdrive')

In [0]:
import pandas as pd
import numpy as np

In [0]:
data = pd.read_csv('prices.csv')

In [7]:
data.shape

(851264, 7)

### Check all columns in the dataset

In [8]:
data.columns

Index(['date', 'symbol', 'open', 'close', 'low', 'high', 'volume'], dtype='object')

### Drop columns `date` and  `symbol`

In [0]:
data.drop(['date','symbol'],axis=1,inplace=True)

In [10]:
data.head()

,open,close,low,high,volume
0,123.430000,125.839996,122.309998,126.250000,2163600.0
1,125.239998,119.980003,119.940002,125.540001,2386400.0
2,116.379997,114.949997,114.930000,119.739998,2489500.0
3,115.480003,116.620003,113.500000,117.440002,2006300.0
4,117.010002,114.970001,114.089996,117.330002,1408600.0


### Consider only first 1000 rows in the dataset for building feature set and target set
Target 'Volume' has very high values. Divide 'Volume' by 1000,000

In [11]:
data.iloc[:1000,:].shape

(1000, 5)

In [0]:
data=data.iloc[:1000,:]

In [0]:
data.loc[:,'volume']=data.loc[:,'volume']/1000000

In [14]:
data.head(2)

,open,close,low,high,volume
0,123.430000,125.839996,122.309998,126.250000,2.1636
1,125.239998,119.980003,119.940002,125.540001,2.3864


In [0]:
feature=data.drop('volume',axis=1)
target=data.pop('volume')

In [16]:
feature.head(2)

,open,close,low,high
0,123.430000,125.839996,122.309998,126.250000
1,125.239998,119.980003,119.940002,125.540001


In [17]:
target.head(2)

0    2.1636
1    2.3864
Name: volume, dtype: float64

### Divide the data into train and test sets

In [0]:
from sklearn.model_selection import train_test_split

In [0]:
feature_train, feature_test, target_train, target_test = train_test_split(feature, target, test_size=0.30, random_state=1)

#### Convert Training and Test Data to numpy float32 arrays


In [0]:
feature_train = feature_train.astype('float32')
feature_test = feature_test.astype('float32')
target_train = target_train.astype('float32')
target_test = target_test.astype('float32')

### Normalize the data
You can use Normalizer from sklearn.preprocessing

In [0]:
from sklearn.preprocessing import Normalizer

In [0]:
transform=Normalizer()

In [0]:
feature_train=transform.fit_transform(feature_train)

In [24]:
feature_train.shape

(700, 4)

In [0]:
target_train=np.array([target_train]).transpose()

In [26]:
target_train.shape

(700, 1)

## Building the Model in tensorflow

1.Define Weights and Bias, use tf.zeros to initialize weights and Bias

In [0]:
w = tf.zeros(shape=(4,1))
b = tf.zeros(shape=(1))

2.Define a function to calculate prediction

In [0]:
def prediction(x, w, b):
    
    xw_matmul = tf.matmul(x, w)
    y = tf.add(xw_matmul, b)
    
    return y

3.Loss (Cost) Function [Mean square error]

In [0]:
def loss(y_actual, y_predicted):
    
    diff = y_actual - y_predicted
    sqr = diff*diff
    avg = tf.reduce_mean(sqr)
    
    return avg

4.Function to train the Model

1.   Record all the mathematical steps to calculate Loss
2.   Calculate Gradients of Loss w.r.t weights and bias
3.   Update Weights and Bias based on gradients and learning rate to minimize loss

In [0]:
def train(x, y_actual, w, b, learning_rate=0.01):
    
    #Record mathematical operations on 'tape' to calculate loss
    with tf.GradientTape() as t:
        
        t.watch([w,b])
        
        current_prediction = prediction(x, w, b)
        current_loss = loss(y_actual, current_prediction)
    
    #Calculate Gradients for Loss with respect to Weights and Bias
    dw, db = t.gradient(current_loss,[w, b])
    
    #Update Weights and Bias
    w = w - learning_rate*dw
    b = b - learning_rate*db
    
    return w, b

## Train the model for 100 epochs 
1. Observe the training loss at every iteration
2. Observe Train loss at every 5th iteration

In [31]:
loss(target_train, prediction(feature_train, w, b))

<tf.Tensor: id=13, shape=(), dtype=float32, numpy=225.87958>

In [32]:
for i in range(100):
    
    w, b = train(feature_train, target_train, w, b)
    print('Current Loss on iteration', i, loss(target_train, prediction(feature_train, w, b)).numpy())

Current Loss on iteration 0 223.49039
Current Loss on iteration 1 221.2885
Current Loss on iteration 2 219.2592
Current Loss on iteration 3 217.38904
Current Loss on iteration 4 215.66547
Current Loss on iteration 5 214.07703
Current Loss on iteration 6 212.61313
Current Loss on iteration 7 211.26398
Current Loss on iteration 8 210.0206
Current Loss on iteration 9 208.8747
Current Loss on iteration 10 207.81862
Current Loss on iteration 11 206.84535
Current Loss on iteration 12 205.9484
Current Loss on iteration 13 205.12172
Current Loss on iteration 14 204.3599
Current Loss on iteration 15 203.65776
Current Loss on iteration 16 203.01071
Current Loss on iteration 17 202.41434
Current Loss on iteration 18 201.86472
Current Loss on iteration 19 201.35822
Current Loss on iteration 20 200.89143
Current Loss on iteration 21 200.46123
Current Loss on iteration 22 200.06473
Current Loss on iteration 23 199.69936
Current Loss on iteration 24 199.36256
Current Loss on iteration 25 199.05223
Cu

In [33]:
for i in range(100):
    
    w, b = train(feature_train, target_train, w, b)
    a=np.mod(i,5)
    if a == 0:
      print('Current Loss on iteration', i, loss(target_train, prediction(feature_train, w, b)).numpy()) 

Current Loss on iteration 0 195.41064
Current Loss on iteration 5 195.40785
Current Loss on iteration 10 195.40598
Current Loss on iteration 15 195.40471
Current Loss on iteration 20 195.40382
Current Loss on iteration 25 195.40321
Current Loss on iteration 30 195.40274
Current Loss on iteration 35 195.40245
Current Loss on iteration 40 195.40219
Current Loss on iteration 45 195.40201
Current Loss on iteration 50 195.40186
Current Loss on iteration 55 195.4017
Current Loss on iteration 60 195.40158
Current Loss on iteration 65 195.40144
Current Loss on iteration 70 195.40134
Current Loss on iteration 75 195.40125
Current Loss on iteration 80 195.40114
Current Loss on iteration 85 195.40103
Current Loss on iteration 90 195.40094
Current Loss on iteration 95 195.40085


### Get the shapes and values of W and b

In [34]:
print('Weights:\n', w.numpy())
print('Shape of Weights:\n', w.shape)
print('Bias:\n',b.numpy())
print('Shape of Bias:\n', b.shape)

Weights:
 [[1.325594 ]
 [1.4239926]
 [1.3286834]
 [1.4386029]]
Shape of Weights:
 (4, 1)
Bias:
 [2.7598708]
Shape of Bias:
 (1,)


### Model Prediction on 1st Examples in Test Dataset

In [0]:
feature_test=transform.fit_transform(feature_test)


In [36]:
prediction([feature_test[0]], w, b)

<tf.Tensor: id=8100, shape=(1, 1), dtype=float32, numpy=array([[5.518816]], dtype=float32)>

In [0]:
pred=prediction([feature_test[0]], w, b)

In [38]:
print("This predicted value of %s is " % feature_test[0],  pred)

This predicted value of [0.4995751  0.49997294 0.495142   0.50525856] is  tf.Tensor([[5.518816]], shape=(1, 1), dtype=float32)


## Classification using tf.Keras

In this exercise, we will build a Deep Neural Network using tf.Keras. We will use Iris Dataset for this exercise.

### Load the given Iris data using pandas (Iris.csv)

In [0]:
iris=pd.read_csv('Iris.csv')

In [40]:
iris.head(2)

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,1,5.1,3.5,1.4,0.2,Iris-setosa
1,2,4.9,3.0,1.4,0.2,Iris-setosa


### Target set has different categories. So, Label encode them. And convert into one-hot vectors using get_dummies in pandas.

In [0]:
iris=pd.get_dummies(iris,columns=['Species'])

In [42]:
iris.head(2)

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species_Iris-setosa,Species_Iris-versicolor,Species_Iris-virginica
0,1,5.1,3.5,1.4,0.2,1,0,0
1,2,4.9,3.0,1.4,0.2,1,0,0


### Splitting the data into feature set and target set

In [0]:
x=iris.iloc[:,1:5]
y=iris.iloc[:,5:8]

In [0]:
y.reset_index(drop=True, inplace=True)

In [0]:
train_x, test_x, train_y, test_y = train_test_split(x, y, test_size=0.20,random_state=1)

###  Building Model in tf.keras

Build a Linear Classifier model  <br>
1.  Use Dense Layer  with input shape of 4 (according to the feature set) and number of outputs set to 3<br> 
2. Apply Softmax on Dense Layer outputs <br>
3. Use SGD as Optimizer
4. Use categorical_crossentropy as loss function 

### Model Training 

In [46]:
model = tf.keras.Sequential([
  tf.keras.layers.Dense(10, input_dim=4),
  tf.keras.layers.Dense(3, activation='softmax'),
])
model.add(tf.keras.layers.BatchNormalization())
model.summary()

Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 10)                50        
_________________________________________________________________
dense_1 (Dense)              (None, 3)                 33        
_________________________________________________________________
batch_normalization_v1 (Batc (None, 3)                 12        
Total params: 95
Trainable params: 89
Non-trainable params: 6
_________________________________________________________________


In [0]:
model.compile(loss='categorical_crossentropy',
              optimizer='sgd',
              metrics=['accuracy'])

In [48]:
model.fit(train_x, train_y.values, 
          validation_data=(test_x, test_y.values), 
          epochs=100,
          batch_size=train_x.shape[0])

Train on 120 samples, validate on 30 samples
Instructions for updating:
Use tf.cast instead.
Epoch 1/100
120/120 [==============================] - 0s 2ms/sample - loss: 6.4108 - acc: 0.0417 - val_loss: 1.3548 - val_acc: 0.4000
Epoch 2/100
120/120 [==============================] - 0s 84us/sample - loss: 6.4611 - acc: 0.0833 - val_loss: 4.5731 - val_acc: 0.4333
Epoch 3/100
120/120 [==============================] - 0s 73us/sample - loss: 7.0190 - acc: 0.4083 - val_loss: 4.8653 - val_acc: 0.4333
Epoch 4/100
120/120 [==============================] - 0s 69us/sample - loss: 6.7576 - acc: 0.4500 - val_loss: 4.6154 - val_acc: 0.4333
Epoch 5/100
120/120 [==============================] - 0s 77us/sample - loss: 6.3463 - acc: 0.5417 - val_loss: 4.6303 - val_acc: 0.4333
Epoch 6/100
120/120 [==============================] - 0s 71us/sample - loss: 6.1962 - acc: 0.5750 - val_loss: 4.6494 - val_acc: 0.4333
Epoch 7/100
120/120 [==============================] - 0s 71us/sample - loss: 6.3262 - acc: 

### Model Prediction

In [0]:
prediction = model.predict_classes(test_x[0:1])

In [50]:
prediction

array([1])

In [0]:
species = np.array(['Setosa', 'Versicolor', 'Virginica'])

In [52]:
print("This species should be",species[prediction])

This species should be ['Versicolor']


### Save the Model

In [0]:
model.save_weights('iris_weights.h5')

In [0]:
model.save('iris.h5')

### Build and Train a Deep Neural network with 2 hidden layer  - Optional - For Practice

Does it perform better than Linear Classifier? What could be the reason for difference in performance?